# Simulation result analysis 

- All replicate results are saved separately, check if they share some commonality


## Section I, trajectory analysis

In [2]:
from Config import Config
from usr_func.checkfolder import checkfolder
import numpy as np
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm
from matplotlib.gridspec import GridSpec
from joblib import Parallel, delayed

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 20

filepath = "./npy/analytical/"
print(os.listdir(filepath))

figpath = os.getcwd() + "/../../../../OneDrive - NTNU/MASCOT_PhD/Projects/GOOGLE/Docs/fig/Sim_2DNidelva/Simulator/AnalyticalEIBV/"
print(os.listdir(figpath))

c = Config()
plg_b = c.get_polygon_border()
plg_o = c.get_polygon_obstacle()
num_steps = c.get_num_steps()
num_replicates = c.get_num_replicates()

os.system("ls -lh " + filepath)

['sigma_10_nuggget_04', 'R_000', 'R_038', 'R_007', 'R_031', '.DS_Store', 'R_009', 'R_036', 'R_062', 'R_065', 'R_053', 'R_054', 'R_008', 'R_037', 'R_030', 'R_039', 'R_006', 'R_001', 'R_055', 'R_052', 'R_064', 'R_063', 'R_041', 'R_079', 'R_046', 'R_070', 'R_048', 'R_077', 'R_023', 'R_024', 'R_012', 'R_015', 'R_049', 'R_076', 'R_071', 'R_078', 'R_047', 'R_040', 'R_014', 'R_013', 'R_025', 'R_022', 'R_066', 'R_059', 'R_061', 'R_057', 'R_068', 'R_050', 'R_004', 'R_003', 'R_035', 'R_032', 'R_051', 'R_056', 'R_069', 'R_060', 'R_067', 'R_058', 'R_033', 'R_034', 'R_002', 'R_005', 'R_027', 'R_018', 'R_020', 'R_016', 'R_029', 'R_011', 'R_045', 'R_042', 'R_074', 'R_073', 'R_010', 'R_017', 'R_028', 'R_021', 'R_026', 'R_019', 'R_072', 'R_075', 'R_043', 'R_044', 'Test30']
['R30test']
total 0
drwx------   4 yaolin  staff   128B Mar 10 14:00 R_000
drwx------   4 yaolin  staff   128B Mar 10 14:00 R_001
drwx------   4 yaolin  staff   128B Mar 10 14:00 R_002
drwx------   4 yaolin  staff   128B Mar 10 14:00

0

In [3]:
Simulators = ["SimulatorRRTStar", "SimulatorMyopic2D"]
# sigmas = [1.5, 1., .5, .1]
# nuggets = [.4, .25, .1, .01]

# sigmas = [1.]
# nuggets = [.4]

sigmas = [.1]
nuggets = [.01]

replicates = os.listdir(filepath)
Nrep = 0
for rep in replicates: 
    if rep.startswith("R_"):
        Nrep += 1
print("Number of replicates: ", Nrep)


def make_plots_total(sigma, nugget, simulator):
    
    def plotf(i, title, filename):
        fig = plt.figure(figsize=(40, 24))
        gs = GridSpec(nrows=2, ncols=3)

        def plot_subplot(data, title):
            for j in range(data.shape[0]):
                plt.plot(data[j, :i, 1], data[j, :i, 0], 'k.-', alpha=.4)
                ax.plot(plg_b[:, 1], plg_b[:, 0], 'r-.')
                ax.plot(plg_o[:, 1], plg_o[:, 0], 'r-.')
                ax.set_xlabel("East")
                ax.set_ylabel("North")
                ax.set_title(title)
                ax.set_aspect("equal")

        def plotf_sim(data, steps, ylabel="IBV"):
            N = data.shape[0]

            hx = np.arange(steps)
            ax.errorbar(hx, y=np.mean(data[:, 0, :steps], axis=0), yerr=np.std(data[:, 0, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="EIBV dominant")
            ax.errorbar(hx, y=np.mean(data[:, 1, :steps], axis=0), yerr=np.std(data[:, 1, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="IVR dominant")
            ax.errorbar(hx, y=np.mean(data[:, 2, :steps], axis=0), yerr=np.std(data[:, 2, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="Equal weights")
            plt.legend(loc="lower left")
            plt.xlabel('Time steps')
            plt.ylabel(ylabel)

        ax = fig.add_subplot(gs[0])
        plot_subplot(traj[:, 0, :, :], "EIBV dominant")

        ax = fig.add_subplot(gs[1])
        plot_subplot(traj[:, 1, :, :], "IVR dominant")

        ax = fig.add_subplot(gs[2])
        plot_subplot(traj[:, 2, :, :], "Equal weights")

        ax = fig.add_subplot(gs[3])
        plotf_sim(ibv, i, "IBV")

        ax = fig.add_subplot(gs[4])
        plotf_sim(vr, i, "VR")

        ax = fig.add_subplot(gs[5])
        plotf_sim(rmse, i, "RMSE")

        plt.suptitle(title)

        plt.savefig(filename)
        plt.close("all")

    string = "/sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget)) + simulator + "/"
    traj = np.empty([0, 3, num_steps, 2])
    ibv = np.empty([0, 3, num_steps])
    rmse = np.empty([0, 3, num_steps])
    vr = np.empty([0, 3, num_steps])

    for i in range(Nrep):
        rep = "R_{:03d}".format(i)
        datapath = filepath + rep + string
        r_traj = np.load(datapath + "traj.npy").reshape(1, 3, num_steps, 2)
        r_ibv = np.load(datapath + "ibv.npy").reshape(1, 3, num_steps)
        r_vr = np.load(datapath + "vr.npy").reshape(1, 3, num_steps)
        r_rmse = np.load(datapath + "rmse.npy").reshape(1, 3, num_steps)

        traj = np.append(traj, r_traj, axis=0)
        ibv = np.append(ibv, r_ibv, axis=0)
        vr = np.append(vr, r_vr, axis=0)
        rmse = np.append(rmse, r_rmse, axis=0)

    savefig = figpath[:-1] + string
    checkfolder(savefig)

    for i in tqdm(range(traj.shape[2])):
        plotf(i, "sigma: {:.1f}, nugget: {:.2f}, simulator: {:s}".format(sigma, nugget, simulator), filename=savefig + "P_{:03d}.png".format(i))

Parallel(n_jobs=5)(delayed(make_plots_total)(sigma=sigma, nugget=nugget, simulator=simulator) for sigma in sigmas for nugget in nuggets for simulator in Simulators)


Number of replicates:  80


100%|██████████| 100/100 [02:39<00:00,  1.59s/it]


[None, None]

In [3]:
import os
figpath = os.getcwd() + "/../../../../OneDrive - NTNU/MASCOT_PhD/Projects/GOOGLE/Docs/fig/Sim_2DNidelva/Simulator"
# sigmas = [1., .1]
# nuggets = [.4, .01]

sigmas = [1.5, 1., .5, .1]
nuggets = [.4, .25, .1, .01]

for sigma in sigmas: 
    for nugget in nuggets: 
#         for simulator in Simulators:  
        string = "/sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget))
#                 string = "/sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget)) + simulator + "/"
        mppath = figpath + string
        os.chdir(mppath)
        os.system("ffmpeg -r 5 -i P_%03d.png -vcodec libx264 -crf 20 -pix_fmt yuv420p sim.mp4")
        print(os.listdir(mppath))


            

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

['P_022.png', 'P_036.png', 'P_037.png', 'P_023.png', 'P_035.png', 'P_021.png', 'P_009.png', 'P_008.png', 'P_020.png', 'P_034.png', 'P_018.png', 'P_030.png', 'P_024.png', '.DS_Store', 'P_025.png', 'P_031.png', 'P_019.png', 'P_027.png', 'P_033.png', 'P_032.png', 'P_026.png', 'P_082.png', 'P_096.png', 'P_041.png', 'P_055.png', 'P_069.png', 'P_068.png', 'P_054.png', 'P_040.png', 'P_097.png', 'P_083.png', 'P_095.png', 'P_081.png', 'P_056.png', 'P_042.png', 'P_043.png', 'P_057.png', 'P_080.png', 'P_094.png', 'P_090.png', 'P_084.png', 'P_053.png', 'P_047.png', 'P_046.png', 'P_052.png', 'P_085.png', 'P_091.png', 'P_087.png', 'P_093.png', 'P_078.png', 'P_044.png', 'P_050.png', 'P_051.png', 'P_045.png', 'P_079.png', 'P_092.png', 'P_086.png', 'P_060.png', 'P_074.png', 'P_048.png', 'P_049.png', 'P_075.png', 'P_061.png', 'P_088.png', 'P_077.png', 'P_063.png', 'P_062.png', 'P_076.png', 'P_089.png', 'P_099.png', 'P_072.png', 'P_066.png', 'sim.mp4', 'P_067.png', 'P_073.png', 'P_098.png', 'P_059.png', 

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

['P_022.png', 'P_036.png', 'P_037.png', 'P_023.png', 'P_035.png', 'P_021.png', 'P_009.png', 'P_008.png', 'P_020.png', 'P_034.png', 'P_018.png', 'P_030.png', 'P_024.png', 'P_025.png', 'P_031.png', 'P_019.png', 'P_027.png', 'P_033.png', 'P_032.png', 'P_026.png', 'P_082.png', 'P_096.png', 'P_041.png', 'P_055.png', 'P_069.png', 'P_068.png', 'P_054.png', 'P_040.png', 'P_097.png', 'P_083.png', 'P_095.png', 'P_081.png', 'P_056.png', 'P_042.png', 'P_043.png', 'P_057.png', 'P_080.png', 'P_094.png', 'P_090.png', 'P_084.png', 'P_053.png', 'P_047.png', 'P_046.png', 'P_052.png', 'P_085.png', 'P_091.png', 'P_087.png', 'P_093.png', 'P_078.png', 'P_044.png', 'P_050.png', 'P_051.png', 'P_045.png', 'P_079.png', 'P_092.png', 'P_086.png', 'P_060.png', 'P_074.png', 'P_048.png', 'P_049.png', 'P_075.png', 'P_061.png', 'P_088.png', 'P_077.png', 'P_063.png', 'P_062.png', 'P_076.png', 'P_089.png', 'P_099.png', 'P_072.png', 'P_066.png', 'sim.mp4', 'P_067.png', 'P_073.png', 'P_098.png', 'P_059.png', 'P_065.png', 

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

...

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

['P_022.png', 'P_036.png', 'P_037.png', 'P_023.png', 'P_035.png', 'P_021.png', 'P_009.png', 'P_008.png', 'P_020.png', 'P_034.png', 'P_018.png', 'P_030.png', 'P_024.png', 'P_025.png', 'P_031.png', 'P_019.png', 'P_027.png', 'P_033.png', 'P_032.png', 'P_026.png', 'P_082.png', 'P_096.png', 'P_041.png', 'P_055.png', 'P_069.png', 'P_068.png', 'P_054.png', 'P_040.png', 'P_097.png', 'P_083.png', 'P_095.png', 'P_081.png', 'P_056.png', 'P_042.png', 'P_043.png', 'P_057.png', 'P_080.png', 'P_094.png', 'P_090.png', 'P_084.png', 'P_053.png', 'P_047.png', 'P_046.png', 'P_052.png', 'P_085.png', 'P_091.png', 'P_087.png', 'P_093.png', 'P_078.png', 'P_044.png', 'P_050.png', 'P_051.png', 'P_045.png', 'P_079.png', 'P_092.png', 'P_086.png', 'P_060.png', 'P_074.png', 'P_048.png', 'P_049.png', 'P_075.png', 'P_061.png', 'P_088.png', 'P_077.png', 'P_063.png', 'P_062.png', 'P_076.png', 'P_089.png', 'P_099.png', 'P_072.png', 'P_066.png', 'sim.mp4', 'P_067.png', 'P_073.png', 'P_098.png', 'P_059.png', 'P_065.png', 

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

['P_022.png', 'P_036.png', 'P_037.png', 'P_023.png', 'P_035.png', 'P_021.png', 'P_009.png', 'P_008.png', 'P_020.png', 'P_034.png', 'P_018.png', 'P_030.png', 'P_024.png', 'P_025.png', 'P_031.png', 'P_019.png', 'P_027.png', 'P_033.png', 'P_032.png', 'P_026.png', 'P_082.png', 'P_096.png', 'P_041.png', 'P_055.png', 'P_069.png', 'P_068.png', 'P_054.png', 'P_040.png', 'P_097.png', 'P_083.png', 'P_095.png', 'P_081.png', 'P_056.png', 'P_042.png', 'P_043.png', 'P_057.png', 'P_080.png', 'P_094.png', 'P_090.png', 'P_084.png', 'P_053.png', 'P_047.png', 'P_046.png', 'P_052.png', 'P_085.png', 'P_091.png', 'P_087.png', 'P_093.png', 'P_078.png', 'P_044.png', 'P_050.png', 'P_051.png', 'P_045.png', 'P_079.png', 'P_092.png', 'P_086.png', 'P_060.png', 'P_074.png', 'P_048.png', 'P_049.png', 'P_075.png', 'P_061.png', 'P_088.png', 'P_077.png', 'P_063.png', 'P_062.png', 'P_076.png', 'P_089.png', 'P_099.png', 'P_072.png', 'P_066.png', 'sim.mp4', 'P_067.png', 'P_073.png', 'P_098.png', 'P_059.png', 'P_065.png', 

frame=  100 fps= 14 q=-1.0 Lsize=    2146kB time=00:00:19.40 bitrate= 906.3kbits/s speed=2.63x    
video:2144kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.088806%
[libx264 @ 0x157605500] frame I:1     Avg QP: 3.69  size:343565
[libx264 @ 0x157605500] frame P:56    Avg QP:13.78  size: 29432
[libx264 @ 0x157605500] frame B:43    Avg QP:20.67  size:  4729
[libx264 @ 0x157605500] consecutive B-frames: 27.0% 44.0%  9.0% 20.0%
[libx264 @ 0x157605500] mb I  I16..4: 79.5% 15.9%  4.6%
[libx264 @ 0x157605500] mb P  I16..4:  0.1%  0.0%  0.1%  P16..4:  0.6%  0.1%  0.1%  0.0%  0.0%    skip:99.0%
[libx264 @ 0x157605500] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.7%  0.1%  0.1%  direct: 0.0%  skip:99.1%  L0:46.7% L1:51.8% BI: 1.5%
[libx264 @ 0x157605500] 8x8 transform intra:15.1% inter:14.1%
[libx264 @ 0x157605500] coded y,uvDC,uvAC intra: 6.8% 4.5% 4.3% inter: 0.2% 0.1% 0.1%
[libx264 @ 0x157605500] i16 v,h,dc,p: 90%  9%  1%  0%
[libx264 @ 0x157605500] i8 v,h,dc,d

In [11]:
os.chdir

<function posix.chdir(path)>